Corpus arborés et parsing - 2024/25

In [ ]:
# Checking the environment
!echo $VIRTUAL_ENV
# and...
!which python
!which pip

In [ ]:
# install spacy model
# https://spacy.io/usage
# %%python3 -m spacy download fr_core_news_sm
# ou python au lieu de python3

In [ ]:
with open("frwiki_50.1000.conllu", encoding="utf-8") as file: #file is a TextIOWrapper object.
    data1 = file.readlines() # liste de lignes
    #data = file.read() # chaîne de caractères
    # for line in file:
    #     print(line)

data1[:20]
# data[-20:]

De nombreuses façons de compter les phrases

In [ ]:
# Si le conllu est bien formaté, on peut faire
n_sentences_A = data1.count("\n")
print(f"{n_sentences_A=}")

In [ ]:
# Mais ce n'est pas du toujours le cas, alors...
n_sentences_B = 0
for line in data1:
    if line.startswith("# sent_id"):
        n_sentences_B += 1
print(f"{n_sentences_B=}")

In [ ]:
# On peut séparer les phrases depuis le début

def read_conllu(path):
    """
    Transformer le fichier conllu dans une liste de listes.
    Chaque liste est une phrase, chaque élément est une ligne.
    """
    data = []
    with open(path, encoding="utf-8") as in_stream:
        next(in_stream)
        sent = []
        for line in in_stream:
            if line == "\n":
                data.append(sent)
                sent = []
                continue
            else:
                sent.append(line)
    return data

path = "frwiki_50.1000.conllu"
data2 = read_conllu(path)

for lst in data2[:1]:
    print(lst)

n_sentences_C = len(data2)
print(f"{n_sentences_C=}")

In [ ]:
# On peut vérifier les résultats, même si dans ce cas c'est inutile
assert n_sentences_A == 996 and n_sentences_A == n_sentences_B and n_sentences_B == n_sentences_C

Un dictionnaire est peut-être la meilleure façon de stocker chaque phrase

In [ ]:
# D'autres informations importantes, telles que les métadonnées et les caractéristiques, ne sont pas prises en compte.

def read_conllu(path) :
    """
    Transformer le fichier conllu dans une liste de liste.
    Chaque liste est une phrase, chaque token est un dictionnaire.
    Certains informations importantes, telles que les métadonnées et les caractéristiques, ne sont pas prises en compte.
    """
    data = []

    with open(path, encoding="utf-8") as in_stream:
        sent = []
        for line in in_stream:
            line = line.strip()
            columns = line.split("\t")

            if columns[0].startswith("#"):
                continue
            if "-" in columns[0]:
                continue
            if columns[0]: # càd, s'il n'est pas vide
                sent.append({
                    "idx" : columns[0],
                    "token" : columns[1],
                    "lemma" : columns[2],
                    "upos" : columns[3],
                    "head" : columns[6],
                    "deprel" : columns[7]})
            else:
                data.append(sent)
                sent = []

    return data

path = "frwiki_50.1000.conllu"
data = read_conllu(path)
data[0]

Compter le nombre des tokens et des lemmes

In [ ]:
# Nombre de tokens
n_tokens = 0
for sentence in data:
    n_tokens += len(sentence)
print(n_tokens)

In [ ]:
# Avec une liste en compréhension 
sum([len(sentence) for sentence in data])

In [ ]:
# 🚧 Quelle est la longueur moyenne d'une phrase dans le corpus ?
# À vous...

In [ ]:
# Combien y a-t-il de tokens différents ?
uniq_tokens = set()
for sentence in data:
    for tk in sentence:
        uniq_tokens.add(tk['token'])

print(f"{len(uniq_tokens)=}")

In [ ]:
# Combien y a-t-il de lemmes différents ?
# À vous...

In [ ]:
# Combien y a-t-il de lemmes différents ? Réponse avec une liste en compréhension  
uniq_lemmas = set([tk['lemma'] for sentence in data for tk in sentence])
print(f"{len(uniq_lemmas)=}")

Maintenant, les compter pour de vrai

In [ ]:
# Récupérer la liste de tous tokens

lemmas = []
for sentence in data:
    for tk in sentence:
        if tk['upos'] != "PUNCT":
            lemmas.append(tk['lemma'])

# Avec une liste en compréhension  
#tokens = [tk['token'] for sentence in data for tk in sentence]

counter = {}
for l in lemmas:
    if l in counter:
        counter[l] += 1
    else:
        counter[l] = 1

# On ordonne le dictionnaire et on obtient une liste de tuples ordonnés
sorted_lemmas = sorted(counter.items(), key=lambda item : item[1], reverse=True)
sorted_lemmas[:5]

In [ ]:
# Avec la meilleure bibliothèque de Python

from collections import Counter

counter = Counter(lemmas)
sorted_lemmas = sorted(counter.items(), key=lambda item : item[1], reverse=True)
sorted_lemmas[:5]

In [ ]:
import seaborn as sns

freq = []
labels = []
for lemma in sorted_lemmas[:20]:
    freq.append(lemma[0])
    labels.append(lemma[1])

sns.barplot(x=freq, y=labels)

In [ ]:
# Représentez graphiquement tous les noms et leur fréquence avec un barplot
# À vous

In [ ]:
# Représentez la distribution de partie du discours avec un barplot
# À vous

In [ ]:
# Combien y a-t-il des adjectifs avant et après leur nom?

adj_position = {"before" : 0, "after": 0}

# for sentence in data:
#     for token_i in sentence:
#         if token_i['upos'] == "NOUN":
#             for token_j in sentence:
                # À compléter, réponse en bas...

In [ ]:
adj_position = {"before" : 0, "after": 0}

for sentence in data:
    for token_i in sentence:
        if token_i['upos'] == "NOUN":
            for token_j in sentence:
                if token_j['head'] == token_i['idx'] and token_j['upos'] == "ADJ":
                    if int(token_i['idx']) < int(token_j['idx']):
                        adj_position['after'] += 1
                    else:
                        adj_position['before'] += 1

sns.barplot(adj_position)

Et si je veux tagger et analyser syntaxiquement un texte ?

In [ ]:
# Tokenisation très naive
with open("conte.txt", encoding="utf-8") as file:
    text = file.read()
sentences = text.split(".")
sentences[:3]

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("fr_core_news_sm")

In [ ]:
sent = sentences[1].strip()
doc = nlp(sent)

for token in doc:
    print(token.text, token.pos_) # token.morph, token.lemma_, token.dep_
# upos = [(token.text, token.pos_) for token in doc]
# print(upos)

In [ ]:
#dir(doc)

In [ ]:
for token in doc:
    print(f"token : {token.text}\t deprel : {token.dep_}\t token_head : {token.head.text}")

In [ ]:
# Trouver un verbe avec un sujet    
verbs = set()
for possible_subject in doc:
    if possible_subject.dep_ == "nsubj" and possible_subject.head.pos_ == "VERB":
        verbs.add(possible_subject.head)
print(verbs)

In [ ]:
displacy.render(doc, style='dep')

Exos

In [ ]:
# Analysez avec SpaCy les phrases du conte Le Joueur de flûte de Hamelin. 
# a. Combient y a-t-il de mots ?
# b. Quelle est la phrase la plus longue ?
# c. Quelle est la longueur moyenne de phrases?
# d. Quel est le mot le plus longue ?
# e. Combient y a-t-il de verbes ?
# f. Plottez avec un barplot les dix mots les plus fréquents.